In [1]:
import logging
from collections import namedtuple
import random
from matplotlib import pyplot as plt

In [2]:
PROBLEM_SIZE = 500
POPULATION_SIZE = 5
OFFSPRING_SIZE = 3

NUM_GENERATIONS = 100

In [3]:
Individual = namedtuple('Individual', ['genome', 'fitness'])

#### Generating the Initial Population

In [4]:
population = list()

def problem(N: int, seed=None) -> list:
    random.seed(seed)
    return list(sorted({ 
        tuple(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    }))

prob = problem(PROBLEM_SIZE, seed=42)

In [6]:
def calculate_fitness(genome):
    # fitness is number of distinct elements in genome
    distinct_elements = set()
    weight = 0
    for s, gene in zip(prob, genome):
        # if the particular element should be taken
        if gene == 1:
            distinct_elements.update(s)
            weight += len(s)
    return len(distinct_elements)

def generate_element():
    genome = [random.randint(0, 1) for _ in range(PROBLEM_SIZE)]
    return Individual(genome, calculate_fitness(genome))

initial_population = [generate_element() for _ in range(POPULATION_SIZE)]

initial_population

[Individual(genome=[1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,

#### Mutation and Recombination

In [7]:
def mutation(genome):
    # mutate a random number of elements
    to_flip = random.randint(0, len(genome))
    # flip the bits
    return [1 - genome[i] if i < to_flip else genome[i] for i in range(len(genome))]

def crossover(genome1, genome2):
    # crossover at a random point
    crossover_point = random.randint(0, len(genome1))
    return genome1[:crossover_point] + genome2[crossover_point:]

def tournament(population):
    # select the best individual from a random sample of the population
    participant = max(random.sample(population, k=3), key=lambda x: x.fitness)
    return Individual(participant.genome, participant.fitness)

def create_offspring(population):
    # create offspring using tournament selection and crossover/mutation
    parent1 = tournament(population)
    parent2 = tournament(population)
    child = Individual(crossover(parent1.genome, parent2.genome), None)
    child = Individual(mutation(child.genome), None)
    return child

In [1]:
best_individual = max(initial_population, key=lambda x: x.fitness)

for i in range(NUM_GENERATIONS):
    # create offspring
    offspring = [create_offspring(initial_population) for _ in range(OFFSPRING_SIZE)]
    # calculate fitness
    offspring = [Individual(child.genome, calculate_fitness(child.genome)) for child in offspring]
    fittest_offspring = max(offspring, key=lambda x: x.fitness)
    
    if fittest_offspring.fitness > best_individual.fitness:
        best_individual = fittest_offspring
    
    # select the best individual
    best_individual = max(offspring, key=lambda x: x.fitness)
    # replace the worst individual
    initial_population = sorted(initial_population, key=lambda x: x.fitness)
    initial_population = initial_population[:-1] + offspring
    
    print(f'Generation {i}: {initial_population}')
    
# get the best individual
print(len(best_individual.genome))

NameError: name 'initial_population' is not defined